In [12]:
from dotenv import load_dotenv
import os

# Cargamos el fichero .env
load_dotenv()

True

In [13]:
import mysql.connector

# Connexión
config = {
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'host': os.getenv('DB_HOST'),
    'database': os.getenv('DB_NAME'),
    'port': os.getenv('DB_PORT')
}

# Nos conectamos a la base de datos
conn = mysql.connector.connect(**config)
cursor = conn.cursor()

In [14]:
# 1. ¿Qué porcentaje de pedidos están subautorizados?
query_1 = """
SELECT 
    ROUND((SUM(CASE WHEN purchase_total_price < products_total THEN 1 ELSE 0 END) / COUNT(*)) * 100, 2) AS porcentaje_subautorizados
FROM fake_orders;
"""
cursor.execute(query_1)
response_1 = cursor.fetchone()

print(f"1. ¿Qué porcentaje de pedidos están subautorizados?\nRespuesta: {response_1[0]}%\n")

1. ¿Qué porcentaje de pedidos están subautorizados?
Respuesta: 21.96%



In [15]:
# 2. ¿Qué porcentaje de pedidos se autorizarían correctamente con una autorización incremental (+20%) sobre el monto en el checkout?
query_2 = """
SELECT 
    ROUND((SUM(CASE WHEN purchase_total_price >= products_total * 1.2 THEN 1 ELSE 0 END) / COUNT(*)) * 100, 2) AS porcentaje_autorizados_incremental
FROM fake_orders;
"""
cursor.execute(query_2)
response_2 = cursor.fetchone()

print(f"2. ¿Qué porcentaje de pedidos se autorizarían correctamente con una autorización incremental (+20%) sobre el monto en el checkout?\nRespuesta: {response_2[0]}%\n")

2. ¿Qué porcentaje de pedidos se autorizarían correctamente con una autorización incremental (+20%) sobre el monto en el checkout?
Respuesta: 31.01%



In [16]:
# 3. ¿Hay diferencias cuando se dividen por país?
query_3 = """
SELECT 
    country_code,
    ROUND((SUM(CASE WHEN purchase_total_price < products_total THEN 1 ELSE 0 END) / COUNT(*)) * 100, 2) AS porcentaje_subautorizados,
    ROUND((SUM(CASE WHEN purchase_total_price >= products_total * 1.2 THEN 1 ELSE 0 END) / COUNT(*)) * 100, 2) AS porcentaje_autorizados_incremental
FROM fake_orders
GROUP BY country_code;
"""
cursor.execute(query_3)
response_3 = cursor.fetchall()

print("3. ¿Hay diferencias cuando se dividen por país?")

for row in response_3:
    print(f"País: {row[0]}, Porcentaje subautorizados: {row[1]}%, Porcentaje autorizados incremental: {row[2]}%")
print("\n")

3. ¿Hay diferencias cuando se dividen por país?
País: AR, Porcentaje subautorizados: 21.54%, Porcentaje autorizados incremental: 33.65%
País: BR, Porcentaje subautorizados: 50.00%, Porcentaje autorizados incremental: 50.00%
País: CI, Porcentaje subautorizados: 66.67%, Porcentaje autorizados incremental: 33.33%
País: CL, Porcentaje subautorizados: 12.95%, Porcentaje autorizados incremental: 87.05%
País: CR, Porcentaje subautorizados: 31.44%, Porcentaje autorizados incremental: 20.14%
País: DO, Porcentaje subautorizados: 24.60%, Porcentaje autorizados incremental: 38.00%
País: EC, Porcentaje subautorizados: 17.19%, Porcentaje autorizados incremental: 37.48%
País: EG, Porcentaje subautorizados: 18.60%, Porcentaje autorizados incremental: 34.35%
País: ES, Porcentaje subautorizados: 24.95%, Porcentaje autorizados incremental: 28.55%
País: FR, Porcentaje subautorizados: 28.42%, Porcentaje autorizados incremental: 32.72%
País: GE, Porcentaje subautorizados: 23.26%, Porcentaje autorizados incr

In [17]:
# 4. Para el resto de pedidos que quedarían fuera de la autorización incremental, ¿qué valores serían necesarios para capturar el monto restante?
query_4 = """
SELECT 
    order_id, 
    (products_total * 1.2) - purchase_total_price AS monto_necesario_para_captura
FROM fake_orders
WHERE purchase_total_price < products_total * 1.2;
"""
cursor.execute(query_4)
response_4 = cursor.fetchall()

print("4. Para el resto de pedidos que quedarían fuera de la autorización incremental, ¿qué valores serían necesarios para capturar el monto restante?")

for row in response_4:
    print(f"Pedido ID: {row[0]}, Monto necesario para captura: {row[1]}")
print("\n")

4. Para el resto de pedidos que quedarían fuera de la autorización incremental, ¿qué valores serían necesarios para capturar el monto restante?
Pedido ID: 31724509, Monto necesario para captura: 0.7640000000000011
Pedido ID: 31848304, Monto necesario para captura: 7.212
Pedido ID: 31867130, Monto necesario para captura: 3.5539999999999985
Pedido ID: 31867210, Monto necesario para captura: 21.599999999999998
Pedido ID: 31876071, Monto necesario para captura: 2.7539999999999996
Pedido ID: 31892209, Monto necesario para captura: 3.2880000000000003
Pedido ID: 31903191, Monto necesario para captura: 0.3440000000000001
Pedido ID: 31906571, Monto necesario para captura: 1.9819999999999993
Pedido ID: 31918981, Monto necesario para captura: 17.927999999999997
Pedido ID: 31936061, Monto necesario para captura: 2.110000000000001
Pedido ID: 31949705, Monto necesario para captura: 4.964000000000002
Pedido ID: 31950653, Monto necesario para captura: 0.09600000000000364
Pedido ID: 31953093, Monto nec

In [18]:
# 5. ¿Qué tiendas son las más problemáticas en términos de pedidos y valor monetario?
query_5a = """
SELECT 
    store_address,
    COUNT(order_id) AS numero_pedidos
FROM fake_orders
GROUP BY store_address
ORDER BY numero_pedidos DESC
LIMIT 10;
"""
cursor.execute(query_5a)
response_5a = cursor.fetchall()
print("5. ¿Qué tiendas son las más problemáticas en términos de cantidad de pedidos?")
for row in response_5a:
    print(f"Tienda: {row[0]}, Número de pedidos: {row[1]}")

query_5b = """
SELECT 
    store_address,
    SUM(purchase_total_price) AS valor_total
FROM fake_orders
GROUP BY store_address
ORDER BY valor_total DESC
LIMIT 10;
"""
cursor.execute(query_5b)
response_5b = cursor.fetchall()

print("\n5. ¿Qué tiendas son las más problemáticas en términos de valor monetario?")

for row in response_5b:
    print(f"Tienda: {row[0]}, Valor total: {row[1]}")
print("\n")

5. ¿Qué tiendas son las más problemáticas en términos de cantidad de pedidos?
Tienda: 28671, Número de pedidos: 492
Tienda: 12513, Número de pedidos: 276
Tienda: 14455, Número de pedidos: 252
Tienda: 28675, Número de pedidos: 248
Tienda: 28712, Número de pedidos: 242
Tienda: 28286, Número de pedidos: 214
Tienda: 28669, Número de pedidos: 201
Tienda: 50832, Número de pedidos: 197
Tienda: 67084, Número de pedidos: 183
Tienda: 51363, Número de pedidos: 179

5. ¿Qué tiendas son las más problemáticas en términos de valor monetario?
Tienda: 28669, Valor total: 6376.070000000003
Tienda: 12513, Valor total: 5307.099999999997
Tienda: 28671, Valor total: 4714.970000000004
Tienda: 28712, Valor total: 3781.389999999995
Tienda: 14455, Valor total: 3514.3300000000004
Tienda: 50832, Valor total: 2515.930000000001
Tienda: 27635, Valor total: 2263.8499999999995
Tienda: 10695, Valor total: 2096.8300000000004
Tienda: 9089, Valor total: 2060.1399999999994
Tienda: 29451, Valor total: 2051.2799999999993




In [19]:

# 6. Para los pedidos subautorizados, ¿hay una correlación entre la diferencia en los precios y la cancelación del pedido?
query_6 = """
SELECT 
    AVG(products_total - purchase_total_price) AS avg_diferencia_precio,
    AVG(CASE WHEN final_status = 'cancelled' THEN 1 ELSE 0 END) AS probabilidad_cancelacion
FROM fake_orders
WHERE purchase_total_price < products_total
GROUP BY (products_total - purchase_total_price);
"""
cursor.execute(query_6)
response_6 = cursor.fetchall()

print("6. Para los pedidos subautorizados, ¿hay una correlación entre la diferencia en los precios y la cancelación del pedido?")

for row in response_6:
    print(f"Diferencia promedio de precio: {row[0]}, Probabilidad de cancelación: {row[1]}")
print("\n")

# Close the cursor and connection
cursor.close()
conn.close()

6. Para los pedidos subautorizados, ¿hay una correlación entre la diferencia en los precios y la cancelación del pedido?
Diferencia promedio de precio: 6.01, Probabilidad de cancelación: 0.0000
Diferencia promedio de precio: 0.00999999999999801, Probabilidad de cancelación: 0.0000
Diferencia promedio de precio: 18.0, Probabilidad de cancelación: 0.0000
Diferencia promedio de precio: 0.17999999999999972, Probabilidad de cancelación: 0.0000
Diferencia promedio de precio: 14.94, Probabilidad de cancelación: 0.0000
Diferencia promedio de precio: 0.9500000000000028, Probabilidad de cancelación: 0.0000
Diferencia promedio de precio: 5.650000000000006, Probabilidad de cancelación: 0.0000
Diferencia promedio de precio: 6.03, Probabilidad de cancelación: 0.0000
Diferencia promedio de precio: 1.6400000000000001, Probabilidad de cancelación: 0.0000
Diferencia promedio de precio: 6.15, Probabilidad de cancelación: 0.0000
Diferencia promedio de precio: 11.35, Probabilidad de cancelación: 0.0000
Dif